# **This is the collaborative code for the group project QTEM Data Challenge**

# ____ __________________ _________ _______ __________ ______________ 

## 1. Install and import packages

In [35]:
from pandas_profiling import ProfileReport
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as smp
from scipy.special import erfinv
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit, Probit, MNLogit
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

## 2. Importing data and defining datasets

In [36]:
%%time
df_Calls = pd.read_csv("b. CARTIER_CALLS.csv")
df_Clienteling = pd.read_csv("c. CARTIER_CLIENTELING.csv")
df_Livechat = pd.read_csv("d. CARTIER_LIVECHAT.csv")
df_PrevSales = pd.read_csv("e. CARTIER_PREVIOUS_SALES.csv")
df_Sales = pd.read_csv("f. CARTIER_SALES.csv")
df_Wishlist = pd.read_csv("g. CARTIER_WISHLIST.csv")

Wall time: 16 s


## Merging datasets

In [38]:
# Merged sales and previous sales dataset with extra column [in_salesdataset] to indicate to which original dataset 
# the data belong

df_Sales['in_salesdataset']=1
df_PrevSales['in_salesdataset']=0

# Rename column from Sale for the concat of datasets

df_Sales.rename(columns = {'articleA':'ArticleA'},inplace = True)

df_AllSales = pd.concat([df_Sales, df_PrevSales])
df_AllSales = df_AllSales.sort_values(by='ClientID', ascending= True)

#to export a dataframe to excel (for siqi): 

#df_BLABLABLA.to_csv('df_BLABLABLA.csv')

In [39]:
# Filter AllSales to drop sales that happened before 2010

df_AllSalesFilt=df_AllSales.drop(df_AllSales[(df_AllSales['TransactionDate_FYYYY']<2010)].index)

In [62]:
df_AllSalesPlot=df_AllSales[df_AllSales.nb_days_since_last_sale>1460]
print(df_AllSalesRepurchase)

                                  InvoiceHeader   Channel TransactionDate  \
122626                                      NaN  Boutique      2019-12-14   
134321                                      NaN  Boutique      2019-12-14   
125277                                      NaN  Boutique      2019-12-14   
130122                                      NaN  Boutique      2020-01-12   
127757                                      NaN  Boutique      2017-01-07   
...                                         ...       ...             ...   
170014  303707220911003-GEMINI-/BIC/AZRIRBARP00  Boutique      2022-09-11   
450007  303707220911003-GEMINI-/BIC/AZRIRBARP00  Boutique      2022-09-11   
863207  303705200821005-GEMINI-/BIC/AZRIRBARP00  Boutique      2020-08-21   
427661  320403220911002-GEMINI-/BIC/AZRIRBARP00  Boutique      2022-09-11   
424837  321502220911006-GEMINI-/BIC/AZRIRBARP00  Boutique      2022-09-11   

        TransactionDate_FYYYY TransactionCategory            ClientID  \
12

In [40]:
# Filter AllSales to drop transaction that arent Sales

df_AllSalesFilt=df_AllSalesFilt.drop(df_AllSalesFilt[(df_AllSalesFilt['TransactionCategory']!='Sale')].index)

In [56]:
print(df_AllSalesFilt.TransactionDate_FYYYY.unique())
print(df_AllSalesFilt.shape[0])
print(df_AllSales.shape[0])

[2020 2023 2022 2019 2017 2021 2016 2013 2018 2015 2010 2012 2014 2011]
1569700
1569700


In [42]:
#df_AllSales = df_AllSalesFilt

## Renaming and uniformizing some columns

In [43]:
# First let's compare all our column names
datasets = [df_Calls,
df_Clienteling,
df_Livechat,
df_Wishlist,]

# Renaming

df_Clienteling.rename(columns = {'clientID':'ClientID'}, inplace = True)
df_Livechat.rename(columns = {'cLientid':'ClientID'}, inplace = True)
df_Wishlist.rename(columns = {'clientID':'ClientID'}, inplace = True)


#print(list(df_AllSales),"\n\n", list(df_Calls),"\n\n", list(df_Clienteling),"\n\n", list(df_Livechat),"\n\n", list(df_Wishlist))

## Defining a sample dataset

In [44]:
# Define a sample dataset to work with for speed purposes if necessary (3% of size)

#df_AllSalesSample = df_AllSales.head(int(round((len(df_allsales)/(100)),0)))
# How to get a random sample of the dataset ??
#df_AllSalesSample=df_AllSales.sample(frac=0.03)
#print(df_AllSalesSample.head(2))

## 2.1 Cleaning the data

In [45]:
# Remove duplicates
df_AllSalesWithDuplicates = df_AllSales
total_rows_before = df_AllSalesWithDuplicates.shape[0]
total_rows_before

# We have 1 111 279 rows in All_Sales dataset with duplicate data

1569771

In [46]:
# Remove duplicates

df_AllSales.drop_duplicates(inplace=True)

total_rows_after = df_AllSales.shape[0]
duplicates_count = total_rows_before - total_rows_after
duplicates_count

71

In [47]:
# Drop irrelevant columns

df_AllSales = df_AllSales.drop(['Hier_Lev_3_txtA', 'Hier_Lev_4_txtA', 'Hier_Lev_5_txtA'], axis=1)

In [48]:
# Missing values

df_AllSales.isna().sum()

InvoiceHeader                 5582
Channel                          0
TransactionDate                  0
TransactionDate_FYYYY            0
TransactionCategory              0
ClientID                         0
AgeAtTransaction            638287
Gender                        1813
PersonBirthDate             528936
WeddingDate                1407946
SpokenLanguage                1334
WrittenLanguage               1334
FirstSalesDate                 445
FirstTransactionDate           245
ProductCategory               1177
ProductSubCategory            1177
ProductFunction               1177
Turnover                     74074
quantity                         0
seq_sales_trs                    0
nb_days_since_last_sale     458147
PurchasedMarketA                 0
PurchasedRegionA                 0
ResidencyRegionA                 0
ResidencyMarketA                 0
BoutiqueNameA               513011
ResidencyCountryA                6
nationalityA               1358025
ArticleA            

In [55]:
#print(df_AllSales[['InvoiceHeader','ClientID','nb_days_since_last_sale','TransactionDate']][df_AllSales['nb_days_since_last_sale'].isnull()])
df_AllSalesnbdaysNA = df_AllSales[['InvoiceHeader','ClientID','nb_days_since_last_sale','TransactionDate']][df_AllSales['nb_days_since_last_sale'].isnull()]
df_AllSalesnbdaysNA.to_csv('df_AllSalesnbdaysNA.csv')

In [57]:
# Replace Nans in n_days_since_last_sale by 0 if Transaction date is the same as first sale date

#df_AllSales = df_AllSales.assign(Calls=df_AllSales.ClientID.isin(df_Calls.ClientID).astype(int))
df_AllSalesTest = df_AllSales
df_AllSalesTest = df_AllSalesTest.assign(nb_days_since_last_sale=df_AllSalesTest.TransactionDate.is(df_AllSalesTest.FirstSalesDate))
#data2['nb_days_since_last_sale'] = data2['nb_days_since_last_sale'].replace(np.nan, 0)
print(df_AllSalesTest.isna().sum())

SyntaxError: invalid syntax (<ipython-input-57-e03a50fca154>, line 5)

In [12]:
filta = df_AllSales['nb_days_since_last_sale'] > 1460
df_AllSales[filta]['nb_days_since_last_sale'].value_counts().sum()

126045

In [13]:
# Changin DataTypes

df_AllSales['Turnover'] = pd.to_numeric(df_AllSales.Turnover, errors='coerce')
df_AllSales.Turnover.apply(np.round).astype('Int64')
#df_AllSales['AgeAtTransaction'] = df_AllSales['AgeAtTransaction'].astype(int)

122626       66
134321       81
9874        447
125277      480
22358      8205
           ... 
1468692    6368
17119      5525
92232      4709
350821     4023
252156      273
Name: Turnover, Length: 2169246, dtype: Int64

## 3. Quick look at our data

In [14]:
# To see the columns we have 

# Run this line if you wish to be able to visualize all the columns whenever you print out the dataframe

pd.set_option('max_columns', None)

print(list(df_AllSales))
#print(list(anything else you want to see))

df_AllSales.describe()
df_AllSales.info()
df_AllSales.isna().sum()

['InvoiceHeader', 'Channel', 'TransactionDate', 'TransactionDate_FYYYY', 'TransactionCategory', 'ClientID', 'AgeAtTransaction', 'Gender', 'PersonBirthDate', 'WeddingDate', 'SpokenLanguage', 'WrittenLanguage', 'FirstSalesDate', 'FirstTransactionDate', 'ProductCategory', 'ProductSubCategory', 'ProductFunction', 'Turnover', 'quantity', 'seq_sales_trs', 'nb_days_since_last_sale', 'PurchasedMarketA', 'PurchasedRegionA', 'ResidencyRegionA', 'ResidencyMarketA', 'BoutiqueNameA', 'ResidencyCountryA', 'nationalityA', 'ArticleA', 'ProductCollectionA', 'in_salesdataset', 'NationalityA']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2169246 entries, 122626 to 252156
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   InvoiceHeader            object 
 1   Channel                  object 
 2   TransactionDate          object 
 3   TransactionDate_FYYYY    int64  
 4   TransactionCategory      object 
 5   ClientID                 objec

InvoiceHeader                 6542
Channel                          0
TransactionDate                  0
TransactionDate_FYYYY            0
TransactionCategory              0
ClientID                         0
AgeAtTransaction            921604
Gender                        2549
PersonBirthDate             764663
WeddingDate                1948156
SpokenLanguage                2632
WrittenLanguage               2632
FirstSalesDate                1054
FirstTransactionDate           407
ProductCategory              35554
ProductSubCategory           35554
ProductFunction              35554
Turnover                     80314
quantity                         0
seq_sales_trs                    0
nb_days_since_last_sale          0
PurchasedMarketA                 0
PurchasedRegionA                 0
ResidencyRegionA                 0
ResidencyMarketA                 0
BoutiqueNameA               617078
ResidencyCountryA                6
nationalityA               1876584
ArticleA            

## 4. Generating the report

In [15]:
### We can generate a profile to obtain description of our datasets
### This can take some time if running report on large datasets
### We can run the report on our sample dataset

# Profile for Sales Sample

#Profile_AllSalesSample = ProfileReport(df_Allsales, title="Pandas Profiling Report Sales")

# Visualizing the report

#Report to iframe (will open report in notebook)

#Profile_AllSalesSample.to_notebook_iframe()

## Report to HTML (Will open up a new tab)
#Profile_AllSalesSample.to_file(output_file='All Sales Sample report')

## 5. Next we would like to create new columns in [Allsales] to identify wether clientID is present in [Calls], [LiveChat], [Clienteling] 

In [16]:
%%time
# #Defining =1 when ClientID is present in the corresponding datasets
df_AllSales = df_AllSales.assign(Calls=df_AllSales.ClientID.isin(df_Calls.ClientID).astype(int))
df_AllSales = df_AllSales.assign(Clienteling=df_AllSales.ClientID.isin(df_Clienteling.ClientID).astype(int))
df_AllSales = df_AllSales.assign(Livechat=df_AllSales.ClientID.isin(df_Livechat.ClientID).astype(int))
df_AllSales = df_AllSales.assign(Wishlist=df_AllSales.ClientID.isin(df_Wishlist.ClientID).astype(int))


print(df_AllSales.Calls.unique(),
      df_AllSales.Clienteling.unique(),
      df_AllSales.Livechat.unique(),
      df_AllSales.Wishlist.unique())

[0 1] [0 1] [0 1] [0 1]
Wall time: 5.39 s


In [20]:
## Useful if were going to filter on Sales dataset for nb_days > 1460 and then apply filter to corresponding clients in prev_sales

df_Sales = df_Sales.assign(Returning=df_Sales.ClientID.isin(df_PrevSales.ClientID).astype(int))

print(df_Sales.Returning.unique())

[1]


In [22]:
print(df_AllSales.head())

                                  InvoiceHeader   Channel TransactionDate  \
134321                                      NaN  Boutique      2019-12-14   
122626                                      NaN  Boutique      2019-12-14   
125277                                      NaN  Boutique      2019-12-14   
22358   302109220925007-GEMINI-/BIC/AZRIRBARP00  Boutique      2022-09-25   
16553   240303000001309-GEMINI-/BIC/AZRIRBAPE00  Boutique      2019-06-24   

        TransactionDate_FYYYY TransactionCategory            ClientID  \
134321                   2020                Sale  0011i00000UNT9LAAX   
122626                   2020                Sale  0011i00000UNT9LAAX   
125277                   2020                Sale  0011i00000UNT9LAAX   
22358                    2023                Sale  0011i00000UNT9LAAX   
16553                    2020                Sale  0011i00000UNTHbAAP   

        AgeAtTransaction Gender PersonBirthDate WeddingDate  ...  \
134321              56.0   Mal

# 6. Create variable "repurchase_long" & "repurchage_exact" that will indicates if client repurchased after 4 years or more or after 4 years +/- 2 months

In [23]:
## Code
df_AllSales['repurchase_long']='0'
df_AllSales['repurchase_long'] = np.where(df_AllSales['nb_days_since_last_sale']>=1400 ,1,0)

print(df_AllSales['repurchase_long'].value_counts())

0    1021364
1      89844
Name: repurchase_long, dtype: int64


In [29]:

df_AllSales['repurchase_exact4']='0'
df_AllSales['repurchase_exact4'] = np.where((df_AllSales['nb_days_since_last_sale']>=1400) & (df_AllSales['nb_days_since_last_sale']<=1520) ,1,0)
print(df_AllSales['repurchase_exact4'].value_counts())

0    1100653
1      10555
Name: repurchase_exact4, dtype: int64



## What kind of question can we answer with this data? (Brainstorm needed)
### Were the purchases made right after communication? 
#### Compare date of purchase with date of latest interaction
### Which type of communication caused the most repurchases ? 
#### Can we use turnover to evaluate which type of communication made the most revenues?

 
## Clienteling:
### What kind of activity category is most effective
### Make a ranking by activity category and type
### Any link between activity status and repurchase / sale amount? 

## Wishlist:
### Do clients with a wishlist repurchase after 4 years? 
### Any link between last modified date, created date and purchase date?

## All Sales
### Make a ranking of product categories repurchased after 4 years in order of magnitude
### Make another table with a column to identify if product repurchased is in same category, subcategory,  of first product bought
### Whats up with wedding dates / anniversary dates and repurchase?

## Calls


## Livechat



In [ ]:
# Code

# 7. Where we're heading

# What kind of variables can we construct out of our dataset to include in the logistic regression

In [17]:
## New Column: Difference between wedding date and date of purchase
## Plot collections \ prod category with repurchase = 0, 1 ?
## Plot most of our variables before we include them in our regression 
## Create dummy variables allowing us to reduce dataset
        ## Create a number of products purchased by ClientID
## How can we use turnover to deduce meaningful insight
## Clean data for missing value in nb_days_Since_last_Sale NaNs... we can double check this if client is in both datasets

In [33]:
df_AllSales.describe()

,TransactionDate_FYYYY,AgeAtTransaction,Turnover,quantity,seq_sales_trs,nb_days_since_last_sale,PurchasedMarketA,PurchasedRegionA,ResidencyRegionA,ResidencyMarketA,BoutiqueNameA,ResidencyCountryA,nationalityA,ArticleA,ProductCollectionA,in_salesdataset,NationalityA,repurchase_long,repurchase_exact4
count,1.111208e+06,699135.000000,1.068670e+06,1.111208e+06,1.111208e+06,816072.000000,1.111208e+06,1.111208e+06,1.111208e+06,1.111208e+06,775260.000000,1.111202e+06,211675.000000,1.110055e+06,1.108838e+06,1.111208e+06,421226.000000,1.111208e+06,1.111208e+06
mean,2.020199e+03,39.575933,4.119382e+03,1.019840e+00,5.529539e+00,570.784758,2.503876e+00,2.174152e+00,2.157427e+00,2.682223e+00,64.372781,5.359783e+01,50.372151,2.065734e+05,5.960033e+01,3.494269e-01,62.491038,8.085255e-02,9.498672e-03
std,3.101523e+00,11.062383,3.713946e+04,1.923368e+00,1.343516e+01,874.900452,3.058985e+00,1.120226e+00,1.190743e+00,3.918234e+00,33.047327,4.698384e+01,46.476591,2.119771e+05,2.866219e+01,4.767892e-01,41.059827,2.726087e-01,9.699719e-02
min,2.010000e+03,18.000000,-1.370543e+04,1.000000e+00,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,2.000000,1.000000e+00,1.000000e+00,0.000000e+00,2.000000,0.000000e+00,0.000000e+00
25%,2.019000e+03,31.000000,2.732958e+02,1.000000e+00,1.000000e+00,83.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,34.000000,1.000000e+00,9.000000,1.009010e+05,4.800000e+01,0.000000e+00,39.000000,0.000000e+00,0.000000e+00
50%,2.022000e+03,38.000000,1.441670e+03,1.000000e+00,2.000000e+00,251.000000,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,70.000000,5.600000e+01,44.000000,1.211110e+05,5.700000e+01,0.000000e+00,44.000000,0.000000e+00,0.000000e+00
75%,2.022000e+03,46.000000,4.041087e+03,1.000000e+00,4.000000e+00,672.000000,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,93.000000,1.100000e+02,99.000000,2.593670e+05,7.800000e+01,1.000000e+00,99.000000,0.000000e+00,0.000000e+00
max,2.023000e+03,99.000000,1.086700e+07,9.540000e+02,3.360000e+02,13441.000000,3.400000e+01,9.000000e+00,1.100000e+01,3.200000e+01,119.000000,1.760000e+02,191.000000,7.845250e+05,1.050000e+02,1.000000e+00,191.000000,1.000000e+00,1.000000e+00


In [19]:
# Ideas for future

# Investigate variables deeper
# Link between Sale / Repair / Return ? If clients who ask for a return or repair end up rebuying, should Cartier contact
                                                                                                                #them?
    # Create 0-1 variable indicating if client got a repair \ return
    # Plot number of days since last sale
    
# Multiple sorting
# Reducing dataset
# Convert floats to int, datatypes, etc
# Create column that counts sales per client
# Create a column that computes average turnover per client
# As discussed by amber, some clients dont have n_days_since_last_sale >= 1440 but if we sum their n_days_since_last_

In [31]:
df_AllSales.to_csv('df_AllSalesCleaned.csv')

## Plotting and Statistics: 

In [ ]:
# This filters the dataset

df_AllSalesPlot=df_AllSales[df_AllSales.nb_days_since_last_sale>1460]
print(df_AllSalesRepurchase)

## Logistic Regression

In [124]:
## Data Preparation

#data1=df_AllSales.drop(df_AllSales[(df_AllSales['TransactionCategory']!='Sale')].index
data1 = df_AllSales

In [125]:
print(data1.head())

                                  InvoiceHeader   Channel TransactionDate  \
122626                                      NaN  Boutique      2019-12-14   
134321                                      NaN  Boutique      2019-12-14   
9874    000000010730476-GEMINI-/BIC/AZRIRBARP00  Boutique      2011-06-11   
125277                                      NaN  Boutique      2019-12-14   
22358   302109220925007-GEMINI-/BIC/AZRIRBARP00  Boutique      2022-09-25   

        TransactionDate_FYYYY TransactionCategory            ClientID  \
122626                   2020                Sale  0011i00000UNT9LAAX   
134321                   2020                Sale  0011i00000UNT9LAAX   
9874                     2012              Repair  0011i00000UNT9LAAX   
125277                   2020                Sale  0011i00000UNT9LAAX   
22358                    2023                Sale  0011i00000UNT9LAAX   

        AgeAtTransaction Gender PersonBirthDate WeddingDate SpokenLanguage  \
122626              

In [126]:
#Collect all the needed raw data 
data2=data1[['ClientID','Gender','Channel','AgeAtTransaction','nb_days_since_last_sale','Turnover',
             'TransactionDate','WeddingDate','PersonBirthDate','Calls','Livechat','Clienteling','Wishlist']]
data2=pd.DataFrame(data2)

In [127]:
data2['nb_days_since_last_sale'].value_counts()

0.0        843178
1.0         16810
2.0         12668
3.0         10905
4.0         10271
            ...  
6926.0          1
6927.0          1
6934.0          1
6938.0          1
17486.0         1
Name: nb_days_since_last_sale, Length: 7440, dtype: int64

### Dummy variables creation

In [22]:
# 

#df_LogRegSales = df_AllSales['ClientID', 'Gender','age', 'Channel', 'Calls', 'Livechat', 'Clienteling', 'Wishlist', 'Wedding', 'Birthday']

# y = repurchase [0,1]

# X = ['Gender', 'Channel','Calls', 'Livechat', 'Clienteling', 'Wishlist', 'Wedding', '']

In [ ]:
# Process Repurchase as dummy

dummy_repurchase=pd.get_dummies(data2['repurchase'],prefix='repurchase')
data2=data2.join(dummy_repurchase['repurchase_1'])

In [ ]:
#Pre-process "Channel" as the second dummy indepedent variable
dummy_Channel=pd.get_dummies(data2['Channel'],prefix='Channel')
data2['Channel_Boutique']=dummy_Channel['Channel_Boutique']
data2['Channel_Web']=dummy_Channel['Channel_Web']
#print(data2)

In [ ]:
#Pre-process "Age" as the third indepedent variable through decentralization, #what is the purpose of this intercept?
'''
data2['AgeAtTransaction'].describe()
k=4 
data2['Age_bins'] = pd.cut(data2['AgeAtTransaction'],k,labels=False)
#add constant
data2['intercept']=1.0
data2=data2.dropna(axis=0)
data2.head()
'''

In [ ]:
#Recode all variables to calculate date intervals for Logit model (Might need to make it an integer)
# Wedding Date Variable
data2['WeddingYear'] = data2['WeddingDate'].dt.year
data2['WeddingMonth'] = data2['WeddingDate'].dt.month
data2['WeddingDay'] = data2['WeddingDate'].dt.day
# Transaction Date Var
data2['TrxYear'] = data2['TransactionDate'].dt.year
data2['TrxMonth'] = data2['TransactionDate'].dt.month
data2['TrxDay'] = data2['TransactionDate'].dt.day
# PersonBirth Date Var
data2['BirthYear'] = data2['PersonBirthDate'].dt.year
data2['BirthMonth'] = data2['PersonBirthDate'].dt.month
data2['BirthDay'] = data2['PersonBirthDate'].dt.day

In [42]:
df_onlyAllSales = df_AllSales.drop(df_AllSales[(df_AllSales['TransactionCategory']!='Sale')].index)
df_onlyAllSales['repurchase'].value_counts()

1    1119493
0     601925
Name: repurchase, dtype: int64

In [ ]:
data1=data1.dropna(subset=['nb_days_since_last_sale'])

In [40]:
data1['repurchase'].value_counts()

1    1119493
0     110029
Name: repurchase, dtype: int64

In [44]:
#filt = df_onlyAllSales[]
df_onlyAllSales['nb_days_since_last_sale'].value_counts()

1.0       16810
2.0       12668
0.0       12552
3.0       10905
4.0       10271
          ...  
6786.0        1
6787.0        1
6788.0        1
6789.0        1
8190.0        1
Name: nb_days_since_last_sale, Length: 7261, dtype: int64

In [70]:
#filt_nbnans = df_onlyAllSales['nb_days_since_last_sale'].isna()
df_onlyAllSales['nb_days_since_last_sale'].isna().value_counts()

False    1229522
True      491896
Name: nb_days_since_last_sale, dtype: int64

In [84]:
df_onlyAllSales[filt_nbnans == True]['InvoiceHeader'].value_counts().head(200)

120699000060061-GEMINI-/BIC/AZRIRBAPE00    26
120699000056756-GEMINI-/BIC/AZRIRBAPE00    23
XJP2540293-GEMINI-/BIC/AZRIRBAJP00         19
XJP104062-GEMINI-/BIC/AZRIRBAJP00          17
303710210206005-GEMINI-/BIC/AZRIRBARP00    16
                                           ..
000000201229382-GEMINI-/BIC/AZRIRBARP00     7
000000018839873-GEMINI-/BIC/AZRIRBARP00     7
000000203903255-GEMINI-/BIC/AZRIRBAAP00     7
000000202050477-GEMINI-/BIC/AZRIRBARP00     7
000000202309831-GEMINI-/BIC/AZRIRBARP00     7
Name: InvoiceHeader, Length: 200, dtype: int64

In [77]:
df_nbSalesNans = df_onlyAllSales[filt_nbnans == True]

In [83]:
df_nbSalesNans['ClientID'].value_counts()
df_nbSalesNans.sort_values('ClientID')


df_nbSalesNans[df_nbSalesNans.ClientID == "0011i00001OZyBhAAL"]

,InvoiceHeader,Channel,TransactionDate,TransactionDate_FYYYY,TransactionCategory,ClientID,AgeAtTransaction,Gender,PersonBirthDate,WeddingDate,SpokenLanguage,WrittenLanguage,FirstSalesDate,FirstTransactionDate,ProductCategory,ProductSubCategory,ProductFunction,Turnover,quantity,seq_sales_trs,nb_days_since_last_sale,PurchasedMarketA,PurchasedRegionA,ResidencyRegionA,ResidencyMarketA,BoutiqueNameA,ResidencyCountryA,nationalityA,ArticleA,Hier_Lev_3_txtA,Hier_Lev_4_txtA,Hier_Lev_5_txtA,ProductCollectionA,in_salesdataset,NationalityA,Calls,Clienteling,Livechat,Wishlist,repurchase
18802,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,RING,1483.33,1,1,NaN,4,4,6,16,53.0,112.0,NaN,108490.0,5.0,157.0,539.0,57.0,0,44.0,0,0,0,0,0
18801,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,NJ,NECK./PEND.,4875.00,1,1,NaN,4,4,6,16,53.0,112.0,NaN,529671.0,61.0,36.0,242.0,13.0,0,44.0,0,0,0,0,0
1234,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,RING,1108.33,1,1,NaN,4,4,6,16,53.0,112.0,NaN,100899.0,5.0,157.0,945.0,104.0,0,44.0,0,0,0,0,0
12994,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,NECK./PEND.,2958.33,1,1,NaN,4,4,6,16,53.0,112.0,NaN,123153.0,5.0,43.0,137.0,74.0,0,44.0,0,0,0,0,0
4466,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,RING,958.33,1,1,NaN,4,4,6,16,53.0,112.0,NaN,108634.0,5.0,157.0,539.0,57.0,0,44.0,0,0,0,0,0
17384,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,BRACELET,5625.00,1,1,NaN,4,4,6,16,53.0,112.0,NaN,121084.0,5.0,157.0,539.0,57.0,0,44.0,0,0,0,0,0
117134,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,RING,958.33,1,1,NaN,4,4,6,16,53.0,112.0,NaN,108622.0,5.0,157.0,539.0,57.0,0,44.0,0,0,0,0,0
132171,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,NECK./PEND.,3583.33,1,1,NaN,4,4,6,16,53.0,112.0,NaN,122809.0,5.0,157.0,539.0,57.0,0,44.0,0,0,0,0,0
120067,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,RING,1041.67,1,1,NaN,4,4,6,16,53.0,112.0,NaN,110942.0,5.0,157.0,945.0,104.0,0,44.0,0,0,0,0,0
21332,120699000060061-GEMINI-/BIC/AZRIRBAPE00,Boutique,2022-02-18,2022,Sale,0011i00001OZyBhAAL,NaN,Male,NaN,NaN,Chinese,Chinese,2022-02-18,2022-02-18,JEWELRY,BIJOUX,RING,1483.33,1,1,NaN,4,4,6,16,53.0,112.0,NaN,108487.0,5.0,157.0,539.0,57.0,0,44.0,0,0,0,0,0


In [1]:
ClientID_grp = df_nbSalesNans.groupby(['ClientID'])

NameError: name 'df_nbSalesNans' is not defined

In [179]:
df_AllSales.Gender.unique()

array(['Male', 'Female', 'Couple', nan, 'Unknown'], dtype=object)

In [ ]:
# Notes from meeting

# figure out why our logit model is wrong
# client model vs transaction model


# better plotting of interesting variables to build a story
# 